In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
estimate=pd.read_csv("/kaggle/input/malnutrition-across-the-globe/malnutrition-estimates.csv")

average=pd.read_csv("/kaggle/input/malnutrition-across-the-globe/country-wise-average.csv")
average.head()

In [ ]:
average.describe()

In [ ]:
x=average.iloc[:,1:]
x

In [ ]:
from sklearn.impute import KNNImputer
knn=KNNImputer()
X=knn.fit_transform(x)
X=pd.DataFrame(X)

In [ ]:
X.columns=["Income Classification","Severe Wasting","Wasting", "Overweight", "Stunting","Underweight","U5 Population ('000s)"]
X.describe()

In [ ]:
country=average.iloc[:,0:1]
X["country"]=country
X

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(15,10))
sns.relplot(data=X,x="Income Classification",y="Overweight")

### Grouping is an option.

In [ ]:
X.groupby(by="Income Classification").mean()[["Overweight"]]

### Lower income level have lower overweight percentages.

In [ ]:
sns.relplot(data=X,x="Income Classification",y="Underweight")

In [ ]:
X.groupby(by="Income Classification").mean()[["Underweight"]]

# İf İncome level increase, Underweight level decrease dramatically. Poor countries really have food supply problems.

In [ ]:
X[X["Income Classification"]<=0]["country"]

### War and Terror and Water Problems and income level have strong correlations.

In [ ]:
X[X["Income Classification"]==3]["country"]

### Trinidad and Tobago, OMAN ???????

In [ ]:
X[X["country"]=="OMAN"]

### Oman is looks like 1 income level country. 

## Okay create decision tree model for Income classification

In [ ]:
y=X["Income Classification"]
xx=X.iloc[:,1:7]
xx

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree=DecisionTreeClassifier(max_depth=3)
dtree.fit(xx,y)
dtree.score(xx,y)

In [ ]:
omanprediction=dtree.predict([[1.68,7.783333,3.55,16.066667,11.916667,332.156]])
print(float(omanprediction))

### Oman goes to second income level.

In [ ]:
from sklearn import tree
plt.figure(figsize=(20,15))
tree.plot_tree(dtree,filled=True,feature_names=xx.columns)
plt.show()

In [ ]:
from eli5.sklearn import explain_decision_tree
explain_decision_tree(dtree,feature_names=list(xx.columns))

### Stunting, Underweight and under five population age is important features.

### Correlation between these features:

In [ ]:
X[["Stunting","Underweight","U5 Population ('000s)"]].corr()

### Stunting and Underweight have stong positive correletion but U5 don't have strong correlation with others

In [ ]:
X[["Stunting","Overweight","U5 Population ('000s)"]].corr()

Overweight not in the same direction with Stunting and U5. Some overweight might actually be a positive income indicator but little indicator.

In [ ]:
X[["Income Classification","Overweight",]].corr()

In [ ]:
X[X["country"].str.contains("turk",case=False)]

Turkey is in very good position.

In [ ]:
X.sort_values(by="Severe Wasting",ascending=True)[0:20]

In [ ]:
X[X["Income Classification"]==2].sort_values(by="Severe Wasting",ascending=True)[0:20]

In [ ]:
X[X["Income Classification"]==2].sort_values(by="Underweight",ascending=True)[0:26]

### Underweight might be bigger problem.